In [1]:
import os
import pandas as pd

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import *
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
print(torch.cuda.get_device_name(0))

Quadro RTX 6000


## Develop model using UCF-Training

### Prepare inputs and labels

In [3]:
train_file_path='../../dataset/UCF/train/'
df_UCF_train=pd.concat([pd.read_pickle(train_file_path+file, compression='gzip') for file in os.listdir(train_file_path)], ignore_index=True)
df_UCF_train['input']= df_UCF_train['TAXPAYER_NAME']+' '+df_UCF_train['mission_spellchk']+' '+df_UCF_train['prgrm_dsc_spellchk']

In [4]:
# Create sentence and encoded label lists
sentences = df_UCF_train.input.values
# labels should be numbers AND continious.
# "input for criterion should satisfy t >= 0 && t < n_classes"
# https://github.com/pytorch/pytorch/issues/1204#issuecomment-326958795
labels = preprocessing.LabelEncoder().fit_transform(df_UCF_train.NTEE1.values)
labels = torch.tensor(labels)

In [5]:
# Import BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in tqdm(sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

100%|██████████| 154424/154424 [05:49<00:00, 441.27it/s]


Original:  DEFENSE TRIAL COUNSEL OF INDIANA EDUCATION OF TRIAL LAWYERS LEGISLATIVE - PROVIDE INFORMATION ON CURRENT LEGAL MATTERS . ; NEWSPAPER - PROVIDE INFORMATION ON PERTINENT TOPICS . ; AMICUS BRIEFS - FILE BRIEFS WITH THE VARIOUS COURTS .
Token IDs: tensor([  101,  3639,  3979,  9517,  1997,  5242,  2495,  1997,  3979,  9559,
         4884,  1011,  3073,  2592,  2006,  2783,  3423,  5609,  1012,  1025,
         3780,  1011,  3073,  2592,  2006,  2566, 10196,  3372,  7832,  1012,
         1025, 26445,  7874, 28760,  1011,  5371, 28760,  2007,  1996,  2536,
         5434,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0, 

In [6]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels, random_state=2018, test_size=0.1)

In [7]:
len(validation_inputs), len(validation_labels), len(validation_masks), len(train_inputs)

(15443, 15443, 15443, 138981)

In [8]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [9]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels=len(df_UCF_train.NTEE1.unique()), # The number of output labels--2 for binary classification.
                                                 # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Create optimizer

In [10]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [11]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [12]:
import numpy as np
import time
import datetime

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

### Start tranining

In [13]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 1000 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch 1,000  of  4,344.    Elapsed: 0:07:52.
  Batch 2,000  of  4,344.    Elapsed: 0:15:44.
  Batch 3,000  of  4,344.    Elapsed: 0:23:37.
  Batch 4,000  of  4,344.    Elapsed: 0:31:29.

  Average training loss: 0.67
  Training epcoh took: 0:34:11

Running Validation...
  Accuracy: 0.87
  Validation took: 0:01:18

======== Epoch 2 / 4 ========
Training...
  Batch 1,000  of  4,344.    Elapsed: 0:07:52.
  Batch 2,000  of  4,344.    Elapsed: 0:15:44.
  Batch 3,000  of  4,344.    Elapsed: 0:23:40.
  Batch 4,000  of  4,344.    Elapsed: 0:31:35.

  Average training loss: 0.40
  Training epcoh took: 0:34:17

Running Validation...
  Accuracy: 0.88
  Validation took: 0:01:18

======== Epoch 3 / 4 ========
Training...
  Batch 1,000  of  4,344.    Elapsed: 0:07:52.
  Batch 2,000  of  4,344.    Elapsed: 0:15:44.
  Batch 3,000  of  4,344.    Elapsed: 0:23:35.
  Batch 4,000  of  4,344.    Elapsed: 0:31:29.

  Average training loss: 0.30
  Training epcoh t

## Evaluate on UCF-Testing

### Prepare inputs and labels

In [15]:
eval_file_path='../../dataset/UCF/test/'
df_UCF_eval=pd.concat([pd.read_pickle(eval_file_path+file, compression='gzip') for file in os.listdir(eval_file_path)], ignore_index=True)
df_UCF_eval['input']= df_UCF_eval['TAXPAYER_NAME']+' '+df_UCF_eval['mission_spellchk']+' '+df_UCF_eval['prgrm_dsc_spellchk']

In [16]:
# Create sentence and encoded label lists
sentences = df_UCF_eval.input.values
# labels should be numbers AND continious.
# "input for criterion should satisfy t >= 0 && t < n_classes"
# https://github.com/pytorch/pytorch/issues/1204#issuecomment-326958795
labels = preprocessing.LabelEncoder().fit_transform(df_UCF_eval.NTEE1.values)
labels = torch.tensor(labels)

In [17]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in tqdm(sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

100%|██████████| 38607/38607 [01:31<00:00, 421.23it/s]


Original:  GREENCROFT GOSHEN INC GREENCROFT GOSDEN PROVIDES ACTIVE , AFFORDABLE RETIREMENT LIVING WITH SUPPORTIVE SERVICES , ASSISTED LIVING , AND SKILLED NURSING CARE . CHARITY CARE FOR EYE JUNE 30 , 2014 AMOUNTED TO $ 4E2793922 . NURSING SERVICES - GREENCROFT GOSDEN IS HEALTH FACILITIES PROVIDE STATE-OF-THE-ART HEALTH CARE TECHNIQUES AND TECHNOLOGY IN AN INVITING AND SUPPORTIVE RESIDENTIAL ENVIRONMENT . OUR SETTING IS APPROPRIATE FOR INDIVIDUALS WHO ARE RECOVERING FROM SURGERY OR ILLNESS , NEED REHABILITATION , OR REQUIRE ONGOING LONG-TERM NURSING CARE . OUR RESTORATIVE CARE PHILOSOPHY REINFORCES THE HEALING AND REHABILITATION PROCESS AND PROMOTES INDEPENDENCE . A FULL RANGE OF ACTIVITIES , CHAPLAINCY , AND SOCIAL SERVICES ARE PROVIDED . WE PROVIDE OUTPATIENT PHYSICAL , OCCUPATIONAL , AND SPEECH/LANGUAGE THERAPIES FOR ALL AGES . WE EMPHASIZE EDUCATION AND COMMUNICATION WITH YOU , YOUR FAMILY , AND REFERRING PHYSICIANS . RESIDENTS WHO DO NOT MISUSE THEIR ASSETS MAY BE ELIGIBLE FOR SUP

In [18]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_data = TensorDataset(input_ids, attention_masks, labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

### Start evaluation

In [21]:
# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
logits_all=[]
label_ids_all=[]

# Evaluate data for one epoch
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    logits_all+=list(np.argmax(logits, axis=1))
    label_ids = b_labels.to('cpu').numpy()
    label_ids_all+=list(label_ids)

    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy

    # Track the number of batches
    nb_eval_steps += 1

# Report the final accuracy for this validation run.
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

  Accuracy: 0.88
  Validation took: 3:19:08


In [22]:
logits_all_letter=preprocessing.LabelEncoder().fit(df_UCF_train.NTEE1.values).inverse_transform(logits_all)
label_ids_all_letter=preprocessing.LabelEncoder().fit(df_UCF_train.NTEE1.values).inverse_transform(label_ids_all)

In [23]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_true=label_ids_all_letter, y_pred=logits_all_letter, digits=4))

                   pre       rec       spe        f1       geo       iba       sup

          A     0.9258    0.9189    0.9908    0.9223    0.9542    0.9039      4291
          B     0.9184    0.9096    0.9839    0.9140    0.9460    0.8883      6419
          C     0.8220    0.8597    0.9959    0.8404    0.9253    0.8446       827
          D     0.9204    0.9391    0.9978    0.9296    0.9680    0.9315      1034
          E     0.8688    0.8470    0.9919    0.8578    0.9166    0.8279      2307
          F     0.7716    0.7716    0.9967    0.7716    0.8770    0.7518       543
          G     0.8327    0.8574    0.9937    0.8449    0.9230    0.8404      1353
          H     0.8144    0.6270    0.9995    0.7085    0.7916    0.6033       126
          I     0.8422    0.8514    0.9969    0.8468    0.9212    0.8363       740
          J     0.8640    0.8083    0.9962    0.8352    0.8973    0.7901      1132
          K     0.8384    0.8448    0.9978    0.8416    0.9181    0.8301       522
   

In [24]:
Counter(logits_all_letter)

Counter({'P': 2401,
         'A': 4259,
         'N': 3962,
         'Y': 1762,
         'G': 1393,
         'E': 2249,
         'C': 865,
         'S': 3551,
         'I': 748,
         'X': 1145,
         'B': 6358,
         'D': 1055,
         'L': 1539,
         'K': 526,
         'M': 1162,
         'R': 240,
         'O': 410,
         'U': 260,
         'Q': 437,
         'F': 543,
         'W': 2075,
         'H': 97,
         'J': 1059,
         'T': 441,
         'V': 70})

In [25]:
t=pd.DataFrame([logits_all_letter, label_ids_all_letter]).T.rename(columns={0:'pred', 1:'true'})
len(t[t.pred==t.true])/len(t)

0.8794260108270521

In [26]:
t.sample(50)

,pred,true
34288,A,A
16221,E,E
26597,N,N
217,L,X
13326,A,A
19788,B,B
505,R,R
9538,Y,Y
15469,A,A
12212,W,W


## Save model

In [27]:
dir_path='../../../bert_model/npo_classifier_mg/'
# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model # Take care of distributed/parallel training
model_to_save.save_pretrained(dir_path)  # save
tokenizer.save_pretrained(dir_path)  # save

('../../../bert_model/npo_classifier_mg/vocab.txt',
 '../../../bert_model/npo_classifier_mg/special_tokens_map.json',
 '../../../bert_model/npo_classifier_mg/added_tokens.json')

### Test saved model

In [29]:
model_loaded = BertForSequenceClassification.from_pretrained(dir_path)  # re-load
tokenizer_loaded = BertTokenizer.from_pretrained(dir_path)  # re-load

In [44]:
# Put the model_loaded in evaluation mode--the dropout layers behave differently
# during evaluation.
model_loaded.cuda()
model_loaded.eval()

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
logits_all=[]
label_ids_all=[]

# Evaluate data for one epoch
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model_loaded not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model_loaded` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_loaded_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model_loaded(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # Get the "logits" output by the model_loaded. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    logits_all+=list(np.argmax(logits, axis=1))
    label_ids = b_labels.to('cpu').numpy()
    label_ids_all+=list(label_ids)

    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy

    # Track the number of batches
    nb_eval_steps += 1

# Report the final accuracy for this validation run.
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

  Accuracy: 0.88
  Validation took: 3:42:34
